Descarga, extracción y análisis del cuestionario ampliado Viviendas del Censo 2020 de la Ciudad de México utilizando Winrar para obtener datos del agua en las viviendas

In [44]:
#Instalar paquetería

if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr, gt, showtext,janitor,sf,httr,jsonlite)
font_add_google("Poppins", "pop")
showtext_auto()

In [45]:
##Borrar datos del entorno
rm(list=ls())

In [46]:
#Opción para que la descarga no se corte
options(timeout=1600)

In [47]:
#URL del Censo 2020
url<-"https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_cdmx_csv.zip"

In [48]:
# Establecer directorio de trabajo y crear carpeta de extracción en "Mis Documentos"
raiz <- file.path(Sys.getenv("USERPROFILE"), "Documents")
dir.create(file.path(raiz, "datoscenso"), showWarnings = F)
directorio <- file.path(raiz, "datoscenso")
setwd(directorio)

In [49]:
#Se establece que, si no existe el archivo viviendas, se descargue el zip de la URL de INEGI
if (!file.exists("Viviendas09.CSV")){
  download.file(url,destfile = "censo.zip",quiet = FALSE, mode="wb")
  #Usaremos Winrar como opción para extracción
  winrar=shQuote("C:/Program Files/WinRAR/WinRAR")
  file="censo.zip"
  dest=directorio
  #Ejecución de la extracción
  cmd=paste(winrar, ' e ',file, ' -ir!*.*','"',dest,'"',sep='')
  system(cmd)
  unlink("datoscenso/censo.zip")
}

In [50]:
# Listar todos los archivos y carpetas dentro del directorio de "datoscenso"
contenido <- list.files(path = directorio, recursive = TRUE, full.names = TRUE)

# Mostrar el contenido
print(contenido)


[1] "C:\\Users\\quetz/Documents/datoscenso/censo.zip"      
[2] "C:\\Users\\quetz/Documents/datoscenso/Migrantes09.CSV"
[3] "C:\\Users\\quetz/Documents/datoscenso/Personas09.CSV" 
[4] "C:\\Users\\quetz/Documents/datoscenso/Viviendas09.CSV"


In [51]:
# Abrir el archivo "Viviendas09.CSV"
archivo_viviendas <- file.path(directorio, "Viviendas09.CSV")

In [52]:
# Verificar si el archivo existe antes de intentar abrirlo
if (file.exists(archivo_viviendas)) {
  # Leer el archivo CSV
  viviendas <- read.csv(archivo_viviendas, header = TRUE, sep = ",", stringsAsFactors = FALSE)
  
  # Mostrar las primeras filas del archivo para verificar la carga
  print(head(viviendas))
} else {
  cat("El archivo Viviendas09.CSV no se encuentra en el directorio.")
  
}


  ENT MUN LOC50K      ID_VIV COBERTURA        ESTRATO UPM FACTOR CLAVIVP
1   9   2      1 90020000001         2 09-002-0001-00   1     27       1
2   9   2      1 90020000002         2 09-002-0001-00   1     27       1
3   9   2      1 90020000003         2 09-002-0001-00   1     27       1
4   9   2      1 90020000004         2 09-002-0001-00   1     27       1
5   9   2      1 90020000005         2 09-002-0001-00   1     27       1
6   9   2      1 90020000006         2 09-002-0001-00   1     27       1
  PAREDES TECHOS PISOS COCINA CUADORM TOTCUART LUG_COC COMBUSTIBLE ESTUFA
1       8     10     3      1       3        4       1           2     NA
2       8     10     3      1       5       10       1           2     NA
3       8     10     3      1       4        7       1           2     NA
4       8     10     2      1       3        4       1           2     NA
5       8     10     2      1       4        5       1           2     NA
6       8     10     2      1       4        

In [53]:
# Definir diseño muestral con nest=TRUE
dm <- viviendas %>%
  as_survey(weights = FACTOR, strata = ESTRATO, ids = UPM, nest = TRUE)

In [54]:
# Evitar notación científica y mostrar números con dos decimales
options(scipen = 999, digits = 2)

In [55]:
###--- Calcular el total de Viviendas en la Ciudad de México
print("Total de viviendas particulares habitadas en la Ciudad de México")
dm%>%
  summarise(viviendas=survey_total()/1000000)

[1] "Total de viviendas particulares habitadas en la Ciudad de México"


viviendas,viviendas_se
<dbl>,<dbl>
2.7,0.076


In [71]:
# Pregunta del censo: AGUA_ENTUBADA


#¿El agua la obtienen de llaves o mangueras que están: dentro de la vivienda?
#sólo en el patio o terreno? ¿No tienen agua entubada?

dm %>%
  mutate(AGUA_ENTUBADA = case_when(
    AGUA_ENTUBADA == 1 ~ "Dentro de la Vivienda",
    AGUA_ENTUBADA == 2 ~ "Sólo en el patio o terreno",
    AGUA_ENTUBADA == 3 ~ "No tienen agua entubada",
    is.na(AGUA_ENTUBADA) ~ "No contestó",  # Corregido
    AGUA_ENTUBADA == 9 ~ "No especificó"
  )) %>%
  group_by(AGUA_ENTUBADA) %>%
  summarise(viviendas = survey_total() / 100000,
            pct = survey_prop()) %>%
  janitor::adorn_totals()

print("El 89.73% de las viviendas de la Ciudad de México tienen agua entubada por dentro de ellas")



AGUA_ENTUBADA,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Dentro de la Vivienda,24.4129,0.7211,0.89738,0.00524
No contestó,0.1008,0.0194,0.00371,0.00071
No especificó,0.0051,0.0029,0.00019,0.00011
No tienen agua entubada,0.2406,0.0329,0.00884,0.00121
Sólo en el patio o terreno,2.4452,0.1452,0.08988,0.00500
Total,27.2046,0.9214,1.00000,0.01227


[1] "El 89.73% de las viviendas de la Ciudad de México tienen agua entubada por dentro de ellas"


In [57]:
# Pregunta del censo: ABA_AGUA_ENTU

#¿El agua que usan en su vivienda proviene: del servicio público de agua? 
#de un pozo comunitario? de un pozo particular? de una pipa? de otra vivienda? de la lluvia? de otro lugar?

dm%>%
  mutate(ABA_AGUA_ENTU=case_when(
    ABA_AGUA_ENTU==1 ~ "Del servicio público de agua",
    ABA_AGUA_ENTU==2 ~ "De un pozo comunitario",
    ABA_AGUA_ENTU==3 ~ "De un pozo particular",
    ABA_AGUA_ENTU==4 ~ "De una pipa",
    ABA_AGUA_ENTU==5 ~ "De otra vivienda",
    ABA_AGUA_ENTU==6 ~ "De la lluvia",
    ABA_AGUA_ENTU==7 ~ "De otra vivienda",
    ABA_AGUA_ENTU==9 ~ "No especificó",))%>%
  group_by(ABA_AGUA_ENTU)%>%
  summarise(viviendas=survey_total()/100000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

print("El 74.81% de las viviendas de la Ciudad de México tienen agua entubada del servicio público del agua")

ABA_AGUA_ENTU,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
De la lluvia,0.0075,0.0044,0.00027,0.00016
De otra vivienda,0.0395,0.0064,0.00145,0.00023
De un pozo comunitario,0.1626,0.0345,0.00598,0.00127
De un pozo particular,0.0132,0.0037,0.00048,0.00014
De una pipa,0.4812,0.0824,0.01769,0.00297
Del servicio público de agua,26.1373,0.7481,0.96077,0.00385
No especificó,0.0168,0.0036,0.00062,0.00013
NA,0.3465,0.0383,0.01274,0.00142
Total,27.2046,0.9215,1.00000,0.01017


[1] "El 74.81% de las viviendas de la Ciudad de México tienen agua entubada del servicio público del agua"


In [58]:
# Pregunta del censo: ABA_AGUA_NO_ENTU

#Entonces, ¿acarrean el agua de: un pozo? una llave comunitaria? otra vivienda? 
#un río, arroyo o lago? ¿La trae una pipa? ¿La captan de la lluvia?

dm%>%
  mutate(ABA_AGUA_NO_ENTU=case_when(
    ABA_AGUA_NO_ENTU==1 ~ "Un pozo",
    ABA_AGUA_NO_ENTU==2 ~ "Una llave comunitariao",
    ABA_AGUA_NO_ENTU==3 ~ "Otra vivienda",
    ABA_AGUA_NO_ENTU==4 ~ "Un río, arroyo o lago",
    ABA_AGUA_NO_ENTU==5 ~ "La trae una pipa",
    ABA_AGUA_NO_ENTU==6 ~ "La captan de la lluvia",
    ABA_AGUA_NO_ENTU==9 ~ "No especificó",))%>%
  group_by(ABA_AGUA_NO_ENTU)%>%
  summarise(viviendas=survey_total()/100000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()




ABA_AGUA_NO_ENTU,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
La captan de la lluvia,0.0022,0.00080,0.000081,0.000029
La trae una pipa,0.1605,0.02820,0.005900,0.001042
Otra vivienda,0.0222,0.00327,0.000816,0.000118
Un pozo,0.0206,0.00701,0.000757,0.000258
"Un río, arroyo o lago",0.0015,0.00098,0.000057,0.000036
Una llave comunitariao,0.0335,0.00697,0.001231,0.000257
NA,26.9640,0.76240,0.991157,0.001213
Total,27.2046,0.80964,1.000000,0.002953


In [61]:
library(dplyr)
library(janitor)
library(glue)

# Calcular el porcentaje y total de viviendas para cada categoría
resultados <- dm %>%
  mutate(ABA_AGUA_NO_ENTU = case_when(
    ABA_AGUA_NO_ENTU == 1 ~ "Un pozo",
    ABA_AGUA_NO_ENTU == 2 ~ "Una llave comunitaria",
    ABA_AGUA_NO_ENTU == 3 ~ "Otra vivienda",
    ABA_AGUA_NO_ENTU == 4 ~ "Un río, arroyo o lago",
    ABA_AGUA_NO_ENTU == 5 ~ "La trae una pipa",
    ABA_AGUA_NO_ENTU == 6 ~ "La captan de la lluvia",
    ABA_AGUA_NO_ENTU == 9 ~ "No especificó"
  )) %>%
  group_by(ABA_AGUA_NO_ENTU) %>%
  summarise(
    viviendas = survey_total() / 100000,  # Total de viviendas
    pct = survey_prop()                   # Porcentaje
  ) %>%
  janitor::adorn_totals()

# Verificar si la creación de resultados fue exitosa
if (exists("resultados")) {
  # Sumar los porcentajes para las fuentes alternativas excluyendo NA
  suma_sin_na <- resultados %>%
    filter(ABA_AGUA_NO_ENTU != "No especificó") %>%
    summarise(suma_pct_sin_na = sum(pct, na.rm = TRUE))

  # Mostrar los porcentajes individuales de cada categoría con total de viviendas
  resultados %>%
    mutate(
      pct = round(pct * 100, 4),  # Redondear porcentaje
      viviendas = round(viviendas, 4),  # Redondear total de viviendas
      print_msg = glue("{ABA_AGUA_NO_ENTU}: {pct}% ({viviendas} viviendas)")
    ) %>%
    arrange(desc(pct)) %>%
    pull(print_msg) %>%
    walk(print)

  # Mostrar el porcentaje acumulado excluyendo "No especificó"
  print(glue("\nSuma de los porcentajes (sin 'No especificó'): {round(suma_sin_na$suma_pct_sin_na * 100, 4)}%"))
} else {
  print("No se pudo crear el objeto 'resultados'.")
}


Total: 100% (27.2046 viviendas)
NA: 99.1157% (26.964 viviendas)
La trae una pipa: 0.59% (0.1605 viviendas)
Una llave comunitaria: 0.1231% (0.0335 viviendas)
Otra vivienda: 0.0816% (0.0222 viviendas)
Un pozo: 0.0757% (0.0206 viviendas)
La captan de la lluvia: 0.0081% (0.0022 viviendas)
Un río, arroyo o lago: 0.0057% (0.0015 viviendas)
Suma de los porcentajes (sin 'No especificó'): 100.8843%


In [63]:
# Pregunta del censo: TINACO

#¿En esta vivienda tienen: tinaco?

dm%>%
  mutate(TINACO=case_when(
    TINACO==1 ~ "Sí",
    TINACO==2 ~ "No",
    TINACO==9 ~ "No especificó",))%>%
  group_by(TINACO)%>%
  summarise(viviendas=survey_total()/100000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

  print("El 82.84% de las viviendas de la Ciudad de México tienen tinaco")

TINACO,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
No,4.550,0.281,0.16727,0.00866
No especificó,0.016,0.006,0.00059,0.00022
Sí,22.537,0.649,0.82844,0.00864
NA,0.101,0.019,0.00371,0.00071
Total,27.205,0.955,1.00000,0.01823


[1] "El 82.84% de las viviendas de la Ciudad de México tienen tinaco"


In [65]:
# Pregunta del censo: CONAGUA

#¿La taza de baño (letrina): tiene descarga directa de agua? le echan agua con cubeta? ¿No se le puede echar agua?

dm%>%
  mutate(CONAGUA=case_when(
    CONAGUA==1 ~ "Tiene descarga directa de agua",
    CONAGUA==2 ~ "Le echan agua con cubeta",
    CONAGUA==3 ~ "No se le puede echar agua",
    CONAGUA==9 ~ "No especificó",))%>%
  group_by(CONAGUA)%>%
  summarise(viviendas=survey_total()/100000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

print("El 86.25% de las viviendas de la Ciudad de México tienen descarga directa de agua en la taza del baño")

CONAGUA,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Le echan agua con cubeta,3.5532,0.1756,0.1306,0.005724
No especificó,0.0028,0.0013,0.0001,0.000048
No se le puede echar agua,0.0274,0.0047,0.0010,0.000173
Tiene descarga directa de agua,23.4629,0.6962,0.8625,0.005836
NA,0.1583,0.0204,0.0058,0.000747
Total,27.2046,0.8983,1.0000,0.012529


[1] "El 86.25% de las viviendas de la Ciudad de México tienen descarga directa de agua en la taza del baño"


In [67]:
# Pregunta del censo: DRENAJE

#¿Esta vivienda tiene drenaje o desagüe conectado a: la red pública? una fosa séptica o 
#tanque séptico (biodigestor)? una tubería que va a dar a una barranca o grieta? 
#una tubería que va a dar a un río, lago o mar? ¿No tiene drenaje?

dm%>%
  mutate(DRENAJE=case_when(
    DRENAJE==1 ~ "La red pública",
    DRENAJE==2 ~ "Una fosa séptica o tanque séptico (biodigestor)",
    DRENAJE==3 ~ "Una tubería que va a dar a una barranca o grieta",
    DRENAJE==4 ~ "Una tubería que va a dar a un río, lago o mar",
    DRENAJE==5 ~ "No tiene drenaje",
    DRENAJE==9 ~ "No especificado",))%>%
  group_by(DRENAJE)%>%
  summarise(viviendas=survey_total()/100000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

print("El 74.76% de las viviendas de la Ciudad de México tienen drenaje a la red pública")

DRENAJE,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
La red pública,25.6051,0.7476,0.94121,0.004987
No especificado,0.0065,0.0036,0.00024,0.000131
No tiene drenaje,0.0430,0.0062,0.00158,0.000227
Una fosa séptica o tanque séptico (biodigestor),1.3628,0.1325,0.05009,0.004802
"Una tubería que va a dar a un río, lago o mar",0.0038,0.0014,0.00014,0.000051
Una tubería que va a dar a una barranca o grieta,0.0826,0.0163,0.00304,0.000600
NA,0.1008,0.0194,0.00371,0.000712
Total,27.2046,0.9269,1.00000,0.011510


[1] "El 74.76% de las viviendas de la Ciudad de México tienen drenaje a la red pública"
